In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model

In [2]:
#parameters
classes = 12
image_size = 224
batch_size= 32
train_total = 3820
validation_total = 930

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = 'imagenet',input_shape=(image_size,image_size,3), alpha=0.5)

In [4]:
x = model_mobilenet.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [5]:
model = Model(inputs=model_mobilenet.input, outputs=x)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 16)      0         
__________

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [66]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 3820 images belonging to 12 classes.
Found 930 images belonging to 12 classes.


In [67]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [68]:
model.load_weights('mobilenet_from_scratch_save_best_final_2_96977.hdf5')

In [69]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.12311724483646617, 0.96344086021505382]

In [12]:
predictions_valid_values = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))
predictions_valid = np.argmax(predictions_valid_values, axis=1)


In [13]:
predictions_valid

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  6,  0,  6,
        6,  0,  0,  5,  0,  0,  4,  6,  6,  0,  0,  0,  0,  4,  0,  0,  0,
        0,  0,  6,  0,  4,  0,  4,  0,  0,  0,  0,  6,  0,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 11,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3

In [14]:
true_positive = np.zeros(12)
false_positive = np.zeros(12)
true_negative = np.zeros(12)
false_negative = np.zeros(12)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [15]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)

true_positive
[  34.   94.   64.  107.   45.   86.  119.   36.  106.   40.   95.   70.]
false_positive
[ 12.   0.   0.   3.   4.   1.  11.   0.   1.   1.   0.   1.]
false_negative
[ 13.   0.   1.   1.   0.   3.  12.   0.   1.   2.   0.   1.]
Total
[  47.   94.   65.  108.   45.   89.  131.   36.  107.   42.   95.   71.]


In [ ]:
# MODEL SMALL FOR BINARY CLASSIFICATION FOR CLASS 0 AND CLASS 6

In [ ]:
#parameters
classes_2 = 2
image_size_2 = 224
batch_size_2 =32
train_total_2 = 739
validation_total_2 = 178

In [ ]:
model_mobilenet_2 = MobileNet(include_top=False, weights = 'imagenet',input_shape=(image_size,image_size,3), alpha=0.5)

In [19]:
x = model_mobilenet_2.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes_2, activation='softmax')(x)

In [20]:
model_2 = Model(inputs=model_mobilenet_2.input, outputs=x)

In [21]:
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 16)      0         
__________

In [22]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [82]:
train_generator_2 = train_datagen.flow_from_directory(
                    'train_2',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator_2 = validation_datagen.flow_from_directory(
                        'validation_2',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 739 images belonging to 2 classes.
Found 178 images belonging to 2 classes.


In [24]:
model_2.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [25]:
callbacks = [ModelCheckpoint(filepath='10_mobilenet_from_scratch_save_best_model_2.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1),
EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./10-metrics_model_2.csv')]

In [58]:
history_2 = model_2.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=callbacks)

Epoch 1/100
Epoch 00000: val_loss improved from inf to 0.50824, saving model to 10_mobilenet_from_scratch_save_best_model_2.hdf5
116s - loss: 0.6648 - acc: 0.6910 - val_loss: 0.5082 - val_acc: 0.7978
Epoch 2/100
Epoch 00001: val_loss did not improve
102s - loss: 0.5347 - acc: 0.7461 - val_loss: 0.5930 - val_acc: 0.6775
Epoch 3/100
Epoch 00002: val_loss improved from 0.50824 to 0.50801, saving model to 10_mobilenet_from_scratch_save_best_model_2.hdf5
104s - loss: 0.4697 - acc: 0.7794 - val_loss: 0.5080 - val_acc: 0.7382
Epoch 4/100
Epoch 00003: val_loss improved from 0.50801 to 0.46528, saving model to 10_mobilenet_from_scratch_save_best_model_2.hdf5
105s - loss: 0.4205 - acc: 0.8026 - val_loss: 0.4653 - val_acc: 0.8067
Epoch 5/100
Epoch 00004: val_loss improved from 0.46528 to 0.43241, saving model to 10_mobilenet_from_scratch_save_best_model_2.hdf5
107s - loss: 0.3741 - acc: 0.8404 - val_loss: 0.4324 - val_acc: 0.7843
Epoch 6/100
Epoch 00005: val_loss improved from 0.43241 to 0.39487,

In [54]:
model_2.load_weights('10_mobilenet_from_scratch_save_best_model_2.hdf5')

In [55]:
model_2.evaluate_generator(validation_generator, int(np.ceil(validation_total_2/batch_size_2)))

[0.32523678661732192, 0.882022469900967]

In [56]:
validation_generator_2 = validation_datagen.flow_from_directory(
                        'validation_2',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 178 images belonging to 2 classes.


In [57]:
predictions_valid_2_values = model_2.predict_generator(validation_generator_2,int(np.ceil(validation_total_2/batch_size_2)))
predictions_valid_2 = np.argmax(predictions_valid_2_values, axis=1)


In [62]:
predictions_valid_2

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0])

In [60]:
true_positive_2 = np.zeros(2)
false_positive_2 = np.zeros(2)
true_negative_2 = np.zeros(2)
false_negative_2 = np.zeros(2)
for i in range(len(predictions_valid_2)):
    if(predictions_valid_2[i]==validation_generator_2.classes[i]):
        true_positive_2[predictions_valid_2[i]] = true_positive_2[predictions_valid_2[i]] + 1
    else:
        false_positive_2[predictions_valid_2[i]] = false_positive_2[predictions_valid_2[i]] + 1
        false_negative_2[validation_generator_2.classes[i]] = false_negative_2[validation_generator_2.classes[i]] +1

In [61]:
print('true_positive_2')
print(true_positive_2)
print('false_positive_2')
print(false_positive_2)
print('false_negative_2')
print(false_negative_2)
print('Total_2')
print(true_positive_2+false_negative_2)

true_positive_2
[  34.  123.]
false_positive_2
[  8.  13.]
false_negative_2
[ 13.   8.]
Total_2
[  47.  131.]


In [115]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [116]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [77]:
indexes = []
for i in range(len(np.argmax(predictions, axis=1))):
    if((np.argmax(predictions, axis=1)[i]==0) or (np.argmax(predictions, axis=1)[i]==6)):
        indexes.append(i)

In [78]:
len(indexes)

157

In [104]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [105]:
predictions_2 = model_2.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids_2 = {train_generator_2.class_indices[x]: x for x in train_generator_2.class_indices}
predicted_classes_2 = [class_ids_2[x] for x in np.argmax(predictions_2, axis=1)]


In [111]:
for i in range(len(indexes)):
    predicted_classes[indexes[i]] = predicted_classes_2[indexes[i]]

{0: 'Black-grass', 1: 'Loose Silky-bent'}

In [96]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_two_models.csv', encoding="utf8", index=False)

In [97]:
from IPython.display import FileLink
FileLink('submission_mobilenet_two_models.csv')

/home/janardhan/plant_seeding/submission_mobilenet_two_models.csv

In [ ]:
# 0.89672